In [ ]:
#from lxml import etree as ET
import shutil
from subprocess import check_output
from pathlib import Path
import numpy as np
import os
#import matplotlib.pyplot as plt
import pandas as pd
#from shapely.geometry import Polygon,
import RClib as RClib



In [ ]:
%load_ext autoreload
%autoreload 2
''
config = RClib.loadconfigs('.\config.json')
scandf = RClib.provide_scandf(inputdirectory=config['workspace'], imageformat = config['baseimageformat'])
previousscandf_path = Path(config['workspace']) / 'rcprocessingdf.pkl'
if previousscandf_path.is_file() and config['resume_processing']:
    previousscandf = pd.read_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))
    previousscandf = previousscandf.set_index('id',drop=False)
    scandf = scandf.set_index('id',drop=False)
    print(previousscandf.columns)
    scandf  = pd.concat([scandf , previousscandf[previousscandf.columns.difference(scandf.columns)]], join='outer', axis=1)
    scandf.update(previousscandf)

print(scandf.columns)

scandf= scandf.apply(RClib.defineRawTherapeeOutput, foldername='02_TIF', axis=1)
scandf= scandf.apply(RClib.defineRealityCaptureOutput, foldername='03_RC', axis=1)
#rcsettings_template = pd.read_csv('.\\allRCsettings_template.csv', sep=';', header=0) 
#print(rcsettings_template)



In [ ]:

scandf2=pd.DataFrame()

for index,scan in scandf.iterrows():
    if not scan['imagedf'] is None:
        if 'pp3file' in scan.keys() and scan['pp3file']:    
            scan['imagedf'] = scan['imagedf'].apply(RClib.developwithRawTherapee, pp3filepath=config['constantpp3'],  outputfolderpath=scan['RToutputfolder'], axis=1) 
        scan = RClib.makeImagelist(scan, imagelistname=scan['id'])
        scan = RClib.createRCproject(scan)    
        if not 'commandlist_alignsettings' in scan['processingstate'] and config['align']:
            scan['commandlist_alignsettings']=Path(config['alignmentsettings'])
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_alignsettings' )
        if not 'align-rccmdpath' in scan['processingstate'] and config['align']:
            scan['commandlist_align'] = []
            scan['commandlist_align'].append('-add ' + str(scan['list_dev-img_path']))
            
            if config['use_gcps']:
                scan['commandlist_align'].append('-detectMarkers')
                if Path(config['constantgcps']).is_file() and Path(config['gcpsparamfile']).is_file():
                    scan['commandlist_align'].append('-importGroundControlPoints ' + str(config['constantgcps']) + ' ' + config['gcpsparamfile'])
                if not Path(config['constantgcps']).is_file() and Path(config['gcpsparamfile']).is_file() and scan['gcpsfile']:
                    scan['commandlist_align'].append('-importGroundControlPoints ' + str(scan['gcpsfile'][0]) + ' ' + config['gcpsparamfile'] )
            scan['commandlist_align'].append('-align')
            scan['commandlist_align'].append('-setReconstructionRegionAuto')
            scan['commandlist_align'].append('-exportReport ' + str(Path(scan['RCoutputfolder'] / Path(scan['id']+'.txt '))) + config['alignmentreport_template'])
            scan['commandlist_align'].append('-save')
            scan['commandlist_align'].append('-quit')
            scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_align', rccmdpathfield='align-rccmdpath')
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='align-rccmdpath' )
        #scan['processingstatus'].append({'alignment_report': Path(scan['RCoutputfolder'] / Path(scan['id']+'.html '))})
        scandf2 = scandf2.append(scan)
#pd.write_csv(Path(config['workspace']) / 'rcprocessingdf.csv', sep=';')
scandf2.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))

In [ ]:
scandf3 = pd.read_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))
for index,scan in scandf3.iterrows():
        if not 'reconstruct-rccmdpath' in scan['processingstate']:
            scan['commandlist_reconstruct'] = []
            scan['commandlist_reconstruct'].append('-calculateNormalModel')  
            scan['commandlist_reconstruct'].append('-save')
            scan['commandlist_reconstruct'].append('-quit')  
            scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_reconstruct', rccmdpathfield='reconstruct-rccmdpath')
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='reconstruct-rccmdpath' )

In [ ]:

def process_results(results):
  # This function will process the results of the GPU-intensive subprocess
  print(f'Processing results: {results}')

def cpu_intensive_function():
  # This function runs the CPU-intensive subprocess
  subprocess.run(['cmd.exe', '/C', 'findstr', 'test', 'large_file.txt'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

# Create a client and connect to the cluster
client = ippclient.Client()
client.block = True

# Create a direct view to the engines
direct_view = client[:]

# Set up the parallel execution
with direct_view.sync_imports():
  import numpy as np

# Create a separate process for the CPU-intensive function
cpu_process = ippcontext.Client().load_balanced_view().apply(cpu_intensive_function)

# Run the GPU-intensive function in a loop
for i in range(10):
  # Run the GPU-intensive function on the engines
  result = direct_view.apply(gpu_intensive_function)

  # Process the results in the background
  process_results(result)

  # Sleep for a short period of time to simulate a long-running process
  time.sleep(1)

# Wait for the CPU-intensive process to finish
cpu_process.wait()

# Disconnect from the cluster
client.close()